<p style="font-size:30px; text-align:center; line-height:120%">
    <br> 
        <b>
        COMS 4995 Applied ML
            Homework 4 
        <br></br>
            Wine Reviews: Task 1
        <br></br>
        </b> 
    <br> 
</p>
<p style="font-size:18px; text-align:left; line-height:120%">
    <br> 
        <b>
        Name: Kirit Dhillon, Sagar Lal
        </b>
    <br> 
        <b>
        Uni: ksd2142, sl3946
        </b>
</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loanding and Exploration

In [2]:
data = pd.read_csv("winemag-data-130k-v2.csv")

# Remove non-usefule columns
data = data.drop(['taster_name', 'taster_twitter_handle'], axis=1)

In [3]:
print(data.dtypes)
print(data.shape)

Unnamed: 0       int64
country         object
description     object
designation     object
points           int64
price          float64
province        object
region_1        object
region_2        object
title           object
variety         object
winery          object
dtype: object
(129971, 12)


### Setup Data

In [4]:
from sklearn.model_selection import train_test_split

#Split before deciding between text and non-text data to allow for joining later
X_trainval, X_test, y_trainval, y_test = train_test_split(data.drop(['points'], axis=1), data['points'])

In [5]:
text_X_trainval = X_trainval['description']
text_X_test = X_test['description']

non_text_X_trainval = X_trainval.drop(['description'], axis=1).reset_index().drop(['index', 'Unnamed: 0'], axis=1)
non_text_X_test = X_test.drop(['description'], axis=1).reset_index().drop(['index', 'Unnamed: 0'], axis=1)

In [6]:
print(text_X_trainval.shape, y_trainval.shape)
print(text_X_test.shape, y_test.shape)
print(non_text_X_trainval.shape)
print(non_text_X_test.shape)

(97478,) (97478,)
(32493,) (32493,)
(97478, 9)
(32493, 9)


## 1.1: Baseline model using non-text features

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [8]:
nt_continuous_features = ['price']
nt_categorical_features = ['country', 'title', 'designation', 'province', 'region_1', 'region_2',
                        'variety', 'winery']

In [9]:
base_categorical_transformer = Pipeline(steps=[('simpleimputer', SimpleImputer(strategy = 'constant', fill_value = "missing")),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

base_continuous_transformer = Pipeline(steps=[('simpleimputer', SimpleImputer(strategy = "median")),
                                         ('scaler', StandardScaler())])

base_continuous_and_categorical_preprocessor = ColumnTransformer(
    transformers=[
        ('num', base_continuous_transformer, nt_continuous_features),
        ('cat', base_categorical_transformer, nt_categorical_features)
    ])

In [23]:
%%time
# Baseline Ridge
base_ridge_clf = Pipeline(steps=[('preprocessor', base_continuous_and_categorical_preprocessor),
                                 ('classifier', Ridge())])

base_ridge_scores = cross_val_score(base_ridge_clf, non_text_X_trainval, y_trainval, cv=5)

print("Baseline Ridge with only CV: ", np.mean(base_ridge_scores))

Baseline Ridge with only CV:  0.5051966649020921
CPU times: user 31.7 s, sys: 6.38 s, total: 38.1 s
Wall time: 16.7 s


In [24]:
%%time
ridge_pipeline = Pipeline(steps=[('preprocessor', base_continuous_and_categorical_preprocessor),
                                ('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, 0.1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=5, return_train_score=True)
grid.fit(non_text_X_trainval, y_trainval)

print(("Baseline Ridge score with GridSearchCV %.2f"
       %grid.score(non_text_X_test, y_test)))
print("Best param:", grid.best_params_)

Baseline Ridge score with GridSearchCV 0.53
Best param: {'regressor__alpha': 1}
CPU times: user 5min 12s, sys: 1min 14s, total: 6min 26s
Wall time: 3min 27s


## 1.2: Text-based model based on:
- Bag-of-Words
- Linear Model

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
bow_vect = CountVectorizer(stop_words="english", max_features=1000)

X_trainval_bow = bow_vect.fit_transform(text_X_trainval)
X_test_bow = bow_vect.transform(text_X_test)

In [16]:
print(X_trainval_bow.shape)
print(X_test_bow.shape)

(97478, 1000)
(32493, 1000)


In [17]:
%%time
ridge_pipeline = Pipeline(steps=[('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, 0.1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(X_trainval_bow, y_trainval)

print(("Text-based Rigde score with GridSearchCV %.2f"
       %grid.score(X_test_bow, y_test)))
print("Best param:", grid.best_params_)

Text-based Rigde score with GridSearchCV 0.61
Best param: {'regressor__alpha': 10}
CPU times: user 4.1 s, sys: 381 ms, total: 4.48 s
Wall time: 4.63 s


## 1.3: More text-based models:
- Explore bigram/unigram models with stemming and removing stopwords
- Expand on that model with using tf-idf

In [18]:
#Stemming function found: http://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/?fbclid=IwAR12cl7yIbpr-TXlS5IGukULm9kR30v_jocnRHAH5efFAECwTLlI7mt_yDw
from textblob import TextBlob

# Use TextBlob
def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

In [20]:
%%time
import warnings
warnings.filterwarnings('ignore')
#unigram/bigram with stemming


bigram_vect = CountVectorizer(stop_words = 'english', tokenizer = textblob_tokenizer, 
                              ngram_range=(1, 2), max_features = 15000) 

X_trainval_bigram = bigram_vect.fit_transform(text_X_trainval)
X_test_bigram = bigram_vect.transform(text_X_test)

ridge_pipeline = Pipeline(steps=[('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, 0.1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(X_trainval_bigram, y_trainval)

print(("Text-based Rigde score with GridSearchCV %.2f"
       %grid.score(X_test_bigram, y_test)))
print("Best param:", grid.best_params_)

Text-based Rigde score with GridSearchCV 0.69
Best param: {'regressor__alpha': 100}
CPU times: user 5min 28s, sys: 15.5 s, total: 5min 43s
Wall time: 5min 35s


In [21]:
print(bigram_vect.vocabulary_)

{'deep': 3857, 'blackberri': 1954, 'syrup': 12675, 'chip': 2910, 'slate': 11631, 'sharp': 11500, 'black-plum': 1947, 'skin': 11625, 'nose': 8890, 'thi': 13111, 'bottl': 2221, 'tannin': 12758, 'veri': 14061, 'chalki': 2671, 'bit': 1868, 'overwhelm': 9495, 'palat': 9532, 'open': 9372, 'reveal': 10815, 'zesti': 14961, 'dens': 3971, 'fruit': 5812, 'touch': 13713, 'peppercorn': 9934, 'solid': 11880, 'choic': 2967, 'better': 1834, 'coupl': 3517, 'year': 14862, 'deep blackberri': 3860, 'blackberri syrup': 2004, 'chip slate': 2911, 'nose thi': 8940, 'thi bottl': 13150, 'tannin veri': 12896, 'palat open': 9652, 'open reveal': 9400, 'dens blackberri': 3974, 'blackberri fruit': 1977, 'fruit touch': 6046, 'coupl year': 3518, 'medium': 8307, 'intens': 7221, 'aroma': 1204, 'tangerin': 12701, 'citru': 3015, 'crush': 3670, 'yellow': 14890, 'flower': 5530, 'transit': 13782, 'flavor': 5164, 'dri': 4147, 'grass': 6430, 'red': 10655, 'appl': 1088, 'mushroom': 8721, 'powder': 10294, 'drink': 4268, 'medium 

In [22]:
%%time
#tf_idf with unigram/bigram with stop words and stemming
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
bigram_tfidf = make_pipeline(bigram_vect,
                             TfidfTransformer(),
                            )

X_trainval_bigram_tfidf = bigram_tfidf.fit_transform(text_X_trainval)

X_test_bigram_tfidf = bigram_tfidf.transform(text_X_test)

ridge_pipeline = Pipeline(steps=[('regressor', Ridge())])

param_grid =  {
               'regressor__alpha': [0.01, 0.1, 1, 10, 100]
              }
grid = GridSearchCV(ridge_pipeline, param_grid, cv=3, return_train_score=True)
grid.fit(X_trainval_bigram_tfidf, y_trainval)

print(("Text-based Rigde score with GridSearchCV %.2f"
       %grid.score(X_test_bigram_tfidf, y_test)))
print("Best param:", grid.best_params_)


Text-based Rigde score with GridSearchCV 0.69
Best param: {'regressor__alpha': 1}
CPU times: user 5min 34s, sys: 9.99 s, total: 5min 44s
Wall time: 6min 20s


## 1.4: Text and non-text features hybrid model

In [31]:
#Convert sparse matrix to df
X_trainval_bigram_tfidf = pd.DataFrame(X_trainval_bigram_tfidf.toarray())

In [32]:
#Merge non-text and BOW and test
full_X_trainval = pd.concat([non_text_X_trainval, X_trainval_bigram_tfidf], axis=1)

In [27]:
#Feed through column transformer and ridge pipeline

full_X_trainval

,country,designation,price,province,region_1,region_2,title,variety,winery,0
0,US,NaN,50.0,California,Santa Ynez Valley,Central Coast,Pegasus Estate 2013 Cabernet Sauvignon (Santa ...,Cabernet Sauvignon,Pegasus Estate,"(0, 14961)\t0.1265726928513257\n (0, 14862)..."
1,South Africa,Made With Organically Grown Grapes,10.0,Western Cape,NaN,NaN,Releaf 2009 Made With Organically Grown Grapes...,Chenin Blanc,Releaf,"(0, 14894)\t0.2607938107305749\n (0, 14890)..."
2,Italy,Vigneto San Martino,NaN,Lombardy,Lugana,NaN,Tommasi 2001 Vigneto San Martino (Lugana),White Blend,Tommasi,"(0, 13324)\t0.1833826092032354\n (0, 13111)..."
3,Portugal,Pomares,15.0,Douro,NaN,NaN,Quinta Nova de Nossa Senhora do Carmo 2011 Pom...,Portuguese Red,Quinta Nova de Nossa Senhora do Carmo,"(0, 14603)\t0.12850168680943805\n (0, 14467..."
4,Italy,NaN,65.0,Tuscany,Brunello di Montalcino,NaN,Villa Poggio Salvi 2003 Brunello di Montalcino,Sangiovese,Villa Poggio Salvi,"(0, 14841)\t0.20026821021310914\n (0, 14840..."
5,US,Unoaked,34.0,California,California,California Other,Passaggio 2014 Unoaked Chardonnay (California),Chardonnay,Passaggio,"(0, 14603)\t0.13027613510096456\n (0, 14467..."
6,South Africa,The Coco,NaN,Western Cape,NaN,NaN,Roos Family 2016 The Coco Merlot-Cabernet Fran...,Merlot-Cabernet Franc,Roos Family,"(0, 14702)\t0.1746941401743838\n (0, 14467)..."
7,France,Grand Vin Blanc Sec,39.0,Bordeaux,Bordeaux Blanc,NaN,Château Fourcas Dupré 2014 Grand Vin Blanc Sec...,Bordeaux-style White Blend,Château Fourcas Dupré,"(0, 14828)\t0.18973753791315043\n (0, 14826..."
8,Argentina,Elevated,13.0,Mendoza Province,Uco Valley,NaN,High Note 2007 Elevated Malbec (Uco Valley),Malbec,High Note,"(0, 13862)\t0.23672106629345946\n (0, 12198..."
9,US,The Nuts Sawyer Lindquist Vineyard,74.0,California,Edna Valley,Central Coast,No Limit 2010 The Nuts Sawyer Lindquist Vineya...,Syrah,No Limit,"(0, 14467)\t0.0443509644664258\n (0, 14158)..."
